# How to use decorators

Learn about the different ways to use tracing and transpilation functions.

⚠️ If you are running this notebook in Colab, you will have to install `Ivy` and some dependencies manually. You can do so by running the cell below ⬇️

If you want to run the notebook locally but don't have Ivy installed just yet, you can check out the [Get Started section of the docs.](https://unify.ai/docs/ivy/overview/get_started.html)

In [1]:
!pip install ivy

## Unify

Firstly, let's create the dummy `numpy` arrays as before: 

In [2]:
# import numpy
import numpy as np
np.random.seed(0)

# create random numpy arrays for testing
x = np.random.uniform(size=10)
mean = np.mean(x)
std = np.std(x)

Let's assume that our target framework is `tensorflow`:

In [3]:
import ivy
import tensorflow as tf
ivy.set_backend("tensorflow")

x = tf.constant(x)

2023-11-01 07:03:38.981554: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/workspaces/ivy/ivy/utils/exceptions.py:390: UserWarning: The current backend: 'tensorflow' does not support inplace updates natively. Ivy would quietly create new arrays when using inplace updates with this backend, leading to memory overhead (same applies for views). If you want to control your memory management, consider doing ivy.set_inplace_mode('strict') which should raise an error whenever an inplace update is attempted with this backend.
  warnings.warn(


In the example below, the `ivy.unify` function is called as a decorator.

In [4]:
import torch

@ivy.unify(source="torch")
def normalize(x):
    mean = torch.mean(x)
    std = torch.std(x)
    return torch.div(torch.sub(x, mean), std)

In [5]:
normalize(x) # unification happens here

ivy.array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])

The function can still be called either *eagerly* or *lazily* when calling as a decorator. The example above is *lazy*, whereas the example below is *eager*:

In [6]:
@ivy.unify(source="torch", args=(x,))
def normalize(x):
    mean = torch.mean(x)
    std = torch.std(x)
    return torch.div(torch.sub(x, mean), std)

In [7]:
normalize(x) # already unified

ivy.array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])

## Trace

In the example below, the `ivy.trace_graph` function is also called as a decorator. (Note that this is now an Ivy function!)

In [8]:
@ivy.trace_graph
def normalize(x):
    mean = ivy.mean(x)
    std = ivy.std(x, correction=1)
    return ivy.divide(ivy.subtract(x, mean), std)

In [9]:
normalize(x) # tracing happens here

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])>

Likewise, the function can still be called either *eagerly* or *lazily* when calling as a decorator. The example above is *lazy*, whereas the example below is *eager*:

In [10]:
@ivy.trace_graph(args=(x,))
def normalize(x):
    mean = ivy.mean(x)
    std = ivy.std(x, correction=1)
    return ivy.divide(ivy.subtract(x, mean), std)

In [11]:
normalize(x) # already traced

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])>

## Transpile

In the example below, the `ivy.transpile` function is called as a decorator.

In [12]:
@ivy.transpile(source="torch", to="tensorflow")
def normalize(x):
    mean = torch.mean(x)
    std = torch.std(x)
    return torch.div(torch.sub(x, mean), std)

In [13]:
normalize(x) # transpilation happens here

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])>

The function can still be called either *eagerly* or *lazily* when calling as a decorator. The example above is *lazy*, whereas the example below is *eager*:

In [14]:
@ivy.transpile(source="torch", to="tensorflow", args=(x,))
def normalize(x):
    mean = torch.mean(x)
    std = torch.std(x)
    return torch.div(torch.sub(x, mean), std)

In [15]:
normalize(x) # already transpiled

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])>

## Round Up

That's it, you now know how `ivy.unify`, `ivy.trace_graph` and `ivy.transpile` can all be used as function decorators! Next, we'll start exploring the transpilation of more involved objects, beginning with libraries 📚